In [1]:
import Cosmology
using Gadfly
using Distributions

In [2]:
# fix Tcmb, Neff
const Tcmb = 2.7255
const Neff = 3.04

3.04

In [3]:
"""
distmod(z, h, OmegaM)

Distance modululus at redshift `z` for a FlatLCDM cosmology with parameters `h` and `OmegaM`.
"""
function distmod(z::Real, h::Real, OmegaM::Real)

    # compute OmegaR
    OmegaG = 4.48131e-7 * Tcmb^4 / h^2
    OmegaN = Neff * OmegaG * (7/8) * (4/11)^(4/3)
    OmegaR = OmegaG + OmegaN

    cosmo = Cosmology.FlatLCDM(h, 1.0 - OmegaM - OmegaR, OmegaM, OmegaR)

    return Cosmology.distmod(cosmo, z)
end

distmod

In [4]:
# Generate some data
h = 0.7
OmegaM = 0.3
M_B = -19.3

# we make our data const so that our likelihood function (below) is type-stable
const N = 100
const z = rand(N)
const mb_err = 0.1 .+ 0.3 .* rand(N)

# add some noise
mb_true = [M_B + distmod(zi, h, OmegaM) for zi in z]
const mb = [rand(Normal(mb_true[i], mb_err[i])) for i in 1:N];

In [ ]:
plot(layer(x=z, y=mb, ymin=mb.-mb_err, ymax=mb.+mb_err, Geom.point, Geom.errorbar))

In [ ]:
# Define log likelihood
function lnlike{T}(x::Vector{T})  # x = [M_B, h, OmegaM]
    s = zero(T)
    for i in 1:N
        mb_model = x[1] + distmod(z[i], x[2], x[3])
        s += ((mb[i] - mb_model) / mb_err[i])^2
    end
    return -0.5 * s
end

## Exact vs approximate gradient of likelihood

In [ ]:
using ForwardDiff
using TimeIt

In [ ]:
ForwardDiff.gradient(lnlike, [-19.1, 0.75, 0.25], Chunk{3}())

In [ ]:
function approx_grad(f, x; dx=sqrt(eps(eltype(x))))
    grad = zeros(x)
    xtmp = copy(x)
    val = f(x)
    for i in 1:length(grad)
        xtmp[i] += dx
        grad[i] = (f(xtmp) - val) / dx
        xtmp[i] -= dx
    end
    
    return grad
end

In [ ]:
approx_grad(lnlike, [-19.1, 0.75, 0.25])